In [7]:
%%writefile C:\Anaconda\Lib\CAMP.py
    
import datetime as dt
import pandas as pd
import numpy as np
import math as math 
import MathsUtilities as MUte

def CalcdHS(Tt,HS,BasePhyllochron):
    """Calculate Haun stage
    Args:
        Tt: Thermal time increment
        HS: Current Haun Stage
        BasePhyllochron: for the current cultivar
    Returns:
        Value to increment Haun stage by
    """
    if HS <= 2.0:
        Phyllochron = float(BasePhyllochron) * 0.75
    else:
        if HS <= 7.0:
            Phyllochron = float(BasePhyllochron)
        else:
            Phyllochron = float(BasePhyllochron) * 1.3
    return Tt/Phyllochron

def CalcdPPVrn(Pp, baseUR, maxUR, dHS):
    """ Calculate delta of upregulation for photo period (Pp) sensitive genes
    
    Args:
        Pp: Photoperiod
        baseUR: dVrn/HS below 8h Pp
        maxUR: dVrn/HS above 16h Pp
        dHS: delta haun stage
    Returns:
        delta Vrn representing the additional expression of that gene today
    """
    if Pp <= 8.0:
        return baseUR * dHS
    if (Pp > 8.0) and (Pp < 16.0):
        return (baseUR + (maxUR * (Pp-8)/(16-8))) * dHS
    if (Pp >= 16.0):
        return maxUR * dHS  

def CalcBaseUpRegVrn1(Tt, dHS, BaseDVrn1):
    """ Calculate upregulation of base Vrn1
    
    Args:
        Tt: Thermal time increment
        dHS: Delta Haun stage
        BaseDVrn1: coeffociennt for Base Vrn1 expression
        
    Returns:
        delta BaseVrn1 representing the additional Vrn1 expression from base expression
    """
    if Tt < 0: 
        BaseDVrn1 = 0
    return BaseDVrn1 * dHS

def CalcColdUpRegVrn1(Tt,dHS, MaxDVrn1, k):
    """ Upregulation of Vrn1 from cold.  Is additional to base vrn1
        BaseDVrn1 in seperate calculation otherwise te same as Brown etal 2013
    Args:
        Tt: Thermal time increment
        dHS: Delta Haun stage
        MaxDVrn1: coefficient for Maximum upregulation of Vrn1
        
    Returns:
        delta ColdVrn1 representing the additional Vrn1 expression from cold upregulation
    """
    UdVrn1 = MaxDVrn1 * np.exp(k*Tt)
    if (Tt < 20):
        return UdVrn1 * dHS
    else:
        return -1

# 
def TSHS(FLN):
    """Haun stage timing of terminal spikelet 
       Inverts equation 5 from Brown etal 2013 FLN =  2.85 + 1.1*TSHS
       Note the intercept differs, was typeo on publication
    Args:
        FLN: The final leaf number
    Returns:
        Estimation of the haun stage timing of terminal spikelet
    """
    return (FLN - 2.85)/1.1

def CAMPmodel(Out, Day, Tt, Pp, Params,
              BasePhyllochron = 120,
              k=-0.18,
              BaseDVrn2=0,
              MaxDVrn3=0.33):
    """ The Cereal Anthesis Molecular Phenology model.
        Based on the ideas presented in Brown etal 2014 (Annals of Botany)
        Alterations made replacing Vrn4 notion with methalation of Vrn1
        Other alterations to implement working code base
    Args:
        Out: "FLN" or None.  If "FLN" will only return estimated FLN else will return full dataframe with daily state variable values
        Day: List, 1:EndDay representing the timesteps in model run
        Tt: List of same length as Day, representing daily temperature
        Pp: List of same length as Day, representing daily photoperiod
        Params: Series with index {BaseDVrn1, MaxDVrn1, MaxDVrn2, BaseDVrn3} and corresponding values for current cultivar 
        BasePhyllochron: for the given cultivar.  Default = 120
        k: Temperature response coefficient for Vrn1 response, Default = -0.18
        BaseDVrn2: Vrn2 expression below 8h Pp.  Default = 0
        MaxDVrn3: Vrn3 expression above 16h Pp.  Default = 0.33
    """
    # Set up Data structure and initialise values
    DF = pd.DataFrame(index = [0], columns = ['Tt','Pp'])
    IsEmerged = False
    IsMethalating = False
    IsCompetent = False
    IsVernalised = False
    IsInduced = False
    IsReproductive = False
    IsAtFlagLeaf = False
    DF.loc[0,'Stage'] = 'Sowing'
    DF.loc[0,'Tt'] = Tt[0]
    DF.loc[0,'Pp'] = Pp[0]
    DF.loc[0,'HS'] = 0.0
    DF.loc[0,'AccumTt'] = 0
    DF.loc[0,'BaseVrn1'] = 0
    DF.loc[0,'ColdVrn1'] = 0
    DF.loc[0,'MethVrn1'] = 0
    DF.loc[0,'Vrn1'] = 0
    DF.loc[0,'Vrn2'] = 0.0
    DF.loc[0,'Vrn3'] = 0.0
    DF.loc[0,'VSHS'] = 0
    DF.loc[0,'FIHS'] = 0
    DF.loc[0,'TSHS'] = 0
    DF.loc[0,'FLN'] = 2.86
    DF.loc[0,'Vrn1Target'] = 1.0
    DF.loc[0,'dHS'] = 0
    DF.loc[0,'dBaseVrn1'] = 0
    DF.loc[0,'dColdVrn1'] = 0
    DF.loc[0,'dMethColdVrn1'] = 0
    DF.loc[0,'dVrn2'] = 0
    DF.loc[0,'dVrn3'] = 0
    d = 0
    Vrn1atVS = 100
    
    # Model daily loop
    while (IsAtFlagLeaf == False) and (d < Day[-1]):
        # Increment day
        d +=1
        # Set daily environment variables
        DF.loc[d,'Tt'] = Tt[d]
        DF.loc[d,'Pp'] = Pp[d]
        #Zero set deltas    
        DF.loc[d,'dBaseVrn1'] = 0
        DF.loc[d,'dColdVrn1'] = 0
        DF.loc[d,'dMethColdVrn1'] = 0
        DF.loc[d,'dVrn2'] = 0
        DF.loc[d,'dVrn3'] = 0
        
        DF.loc[d,'AccumTt'] = DF.loc[d-1,'AccumTt'] + DF.loc[d,'Tt']
        DF.loc[d,'dHS'] = CalcdHS(DF.loc[d,'Tt'],DF.loc[d-1,'HS'],BasePhyllochron)
       
        #Work out base, cold induced Vrn1 expression and methalyation until vernalisation is complete
        if (IsVernalised==False): 
            # If methalated Vrn1 expression is less that Vrn1Target do Vrn1 upregulation
            if(DF.loc[d-1,'MethVrn1'] < DF.loc[d-1,'Vrn1Target']):
                DF.loc[d,'dBaseVrn1'] = CalcBaseUpRegVrn1(DF.loc[d,'Tt'],DF.loc[d,'dHS'],Params.BaseDVrn1) 
                DF.loc[d,'dColdVrn1'] = CalcColdUpRegVrn1(DF.loc[d,'Tt'],DF.loc[d,'dHS'],Params.MaxDVrn1,k)
            # If Vrn1(base + cold) equals Vrn1Target methalate coldVrn1.  BaseVrn1 is all methalated every day
            if (IsMethalating == True):
                DF.loc[0,'dMethColdVrn1'] = max(0.0,min(DF.loc[d,'dColdVrn1'],DF.loc[d-1,'ColdVrn1']))
            # Set todays Vrn1 expression values using deltas just calculated
            DF.loc[d,'BaseVrn1'] = DF.loc[d-1,'BaseVrn1'] + DF.loc[d,'dBaseVrn1']
            DF.loc[d,'MethVrn1'] = DF.loc[d-1,'MethVrn1'] + DF.loc[0,'dMethColdVrn1'] + DF.loc[d,'dBaseVrn1']
            DF.loc[d,'ColdVrn1'] = max(0.0,DF.loc[d-1,'ColdVrn1'] + DF.loc[d,'dColdVrn1'] - DF.loc[0,'dMethColdVrn1'])   
            DF.loc[d,'Vrn1'] = DF.loc[d,{'MethVrn1','ColdVrn1'}].sum()
            
        # Once Vernalisation complete, carry over vern expression values from yesterday
        else:
            DF.loc[d,'BaseVrn1'] = DF.loc[d-1,'BaseVrn1']
            DF.loc[d,'ColdVrn1'] = DF.loc[d-1,'ColdVrn1'] 
            DF.loc[d,'MethVrn1'] = DF.loc[d-1,'MethVrn1'] 
            DF.loc[d,'Vrn1'] = DF.loc[d-1,'Vrn1']
            
        #Then work out Vrn2 expression 
        if (IsVernalised == False) and (IsCompetent==True):
            DF.loc[d,'dVrn2'] = CalcdPPVrn(DF.loc[d,'Pp'], BaseDVrn2, Params.MaxDVrn2, DF.loc[d,'dHS'])
        DF.loc[d,'Vrn2'] = DF.loc[d-1,'Vrn2'] + DF.loc[d,'dVrn2']
        DF.loc[d,'Vrn1Target'] = 1.0 + DF.loc[d,'Vrn2']
            
        #Work out if methalation of cold response has started 
        if (DF.loc[d,'Vrn1']>= DF.loc[d,'Vrn1Target']) and (IsMethalating == False):
            IsMethalating = True
            
        #work out if vernalisation is complete    
        if (DF.loc[d,'MethVrn1'] >= DF.loc[d,'Vrn1Target']) and (IsCompetent >= True) and (IsVernalised==False):
            IsVernalised  = True
            DF.loc[d,'Stage'] = 'Vern Sat'
            Vrn1atVS = DF.loc[d,'MethVrn1']
            
        # Downregulate vrn1 expression if we are at Vrn1target
        if(IsMethalating==True) and (IsVernalised==False) and (DF.loc[d,'dColdVrn1']>0):
            dDRVrn1 = DF.loc[d,'Vrn1'] - DF.loc[d,'Vrn1Target']
            dDRColdVrn1 = min(dDRVrn1,DF.loc[d,'dColdVrn1'] + DF.loc[d,'dBaseVrn1'],DF.loc[d,'ColdVrn1'])
            DF.loc[d,'ColdVrn1'] = DF.loc[d,'ColdVrn1'] - dDRColdVrn1 
        
    #Then work out Vrn3 expression
        if (IsVernalised == True) and (IsCompetent==True) and (IsReproductive == False):
            DF.loc[d,'dVrn3'] = CalcdPPVrn(DF.loc[d,'Pp'], Params.BaseDVrn3, MaxDVrn3, DF.loc[d,'dHS'])
        DF.loc[d,'Vrn3'] = min(1,DF.loc[d-1,'Vrn3'] + DF.loc[d,'dVrn3'])
        
    #Then add Vrn3 expression effects to Vrn1 upregulation
        DF.loc[d,'Vrn1'] = DF.loc[d,'Vrn1'] + DF.loc[d,'dVrn3']

    #Then work out phase progression based on Vrn expression
        if (DF.loc[d,'Vrn1'] >= (Vrn1atVS + 0.3)) and (IsInduced == False):
            IsInduced = True;
        if (DF.loc[d,'Vrn1'] >=  (Vrn1atVS + 1.0)) and (IsReproductive == False):
            IsReproductive = True;
            DF.loc[d,'Stage'] = 'Term Spike'
        
    #Finally do Leaf appearance calculations   
        if (DF.loc[d,'AccumTt'] >= BasePhyllochron) and (IsEmerged==False):
            IsEmerged = True
            DF.loc[d,'Stage'] = 'Emergence'        
        if IsEmerged == True:
            DF.loc[d,'HS'] = DF.loc[d-1,'HS'] +  DF.loc[d-1,'dHS']
        else:
            DF.loc[d,'HS'] = 0
        if (DF.loc[d,'HS'] >= 1.1) and (IsCompetent==False):
            IsCompetent=True
            DF.loc[d,'Stage'] = 'Competense'     
    
    #Set Haun stage variables
        if IsVernalised == False:
            DF.loc[d,'VSHS'] = DF.loc[d,'HS']
        else:
            DF.loc[d,'VSHS'] = DF.loc[d-1,'VSHS']
        if IsInduced == False:
            DF.loc[d,'FIHS'] = DF.loc[d,'HS']
        else: 
            DF.loc[d,'FIHS'] = DF.loc[d-1,'FIHS']
        if IsReproductive == False:
            DF.loc[d,'TSHS'] = DF.loc[d,'HS']
            DF.loc[d,'FLN'] = 2.86 + 1.1 * DF.loc[d,'TSHS']
        else:
            DF.loc[d,'TSHS'] = DF.loc[d-1,'TSHS']
            DF.loc[d,'FLN'] = DF.loc[d-1,'FLN']
    #Work out if Flag leaf has appeared.
        if DF.loc[d,'HS'] >= DF.loc[d,'FLN']:
            IsAtFlagLeaf = True    
            DF.loc[d,'Stage'] = 'FlagLeaf'
    
    #Add states to dataframe
        DF.loc[d,'IsEmerged'] = IsEmerged
        DF.loc[d,'IsCompetent'] = IsCompetent
        DF.loc[d,'IsVernalised'] = IsVernalised
        DF.loc[d,'IsInduced'] = IsInduced
        DF.loc[d,'IsReproductive'] = IsReproductive
        DF.loc[d,'IsAtFlagLeaf'] = IsAtFlagLeaf
    
    if (Out == 'FLN') and (IsAtFlagLeaf == True):
        return DF.iloc[-1,:]['FLN']
    else:
        return DF
    
#List of parameter names that CalcCultivarVrnCoeffs calculates
CultivarParamList = ['TSHS_LV','TSHS_LN', 'TSHS_SV', 'TSHS_SN', 
                     'VernTreatHS','EmergHS',
                     'PPS','MaxDVrn3','BaseDVrn3',                         
                     'VSHS_SN','BaseDVrn1',
                     'VSHS_LN','L_Vrn1TargetInc','MaxDVrn2',
                     'VSHS_SV','TransToVSHS_SV',
                     'MethVern1@Trans','BaseVrn1@Trans','MethColdVrn1@Trans',
                     'RelativeMethTiming','BaseVern1@Meth',
                     'DVrn1@Tt','MaxDVrn1']

def CalcCultivarVrnCoeffs(FLN_LV, FLN_LN, FLN_SV, FLN_SN,
                          TreatmentTtDuration = 90, BasePhyllochron = 120, 
                          Tt = 1, k = -0.18, TtEmerge = 90):
    """ Calculate VrnRate paremeters for genotype from given FLN observations
    Args:
        FLN_LV: Final Leaf Number under long day full vernalisation conditions
        FLN_LN: Final Leaf Number under long day un-vernalisation conditions
        FLN_SV: Final Leaf Number under short day full vernalisation conditions
        FLN_SN: Final Leaf Number under short day un-vernalisation conditions
        TreatmentTtDuration: ThermalTime Duration of vernalisation treatment, Default = 90oCd (12 weeks at 1oC)
        BasePhyllochron: For given cultivar, Default = 120
        Tt: Vernalisation Treatment Temperature, Default = 1oC
        k: Vernalisation temperature response coefficient, default = -0.19
        TtEmerge: Thermal time from sowing to emergence, degault 90oCd (assuming one phyllochron to emerge)
    Returns:
        Series with values assigned to each variable in CultivarParameterList 
    """
    #Set up Data Store
    data = pd.Series(index = CultivarParamList)
    
    # Haun stage duration of vernalisation treatment period
    data['VernTreatHS'] = TreatmentTtDuration/(BasePhyllochron*0.75) 
    # Haun stage equivelents from imbibing to Emergence
    data['EmergHS'] = TtEmerge/(BasePhyllochron*0.75)
    # Haun stage duration from vernalisation saturation to terminal spikelet under long day conditions
    L_HSVsTs = 3.0
    # Haun stage at which plant becomes competent to respond to Pp (this is required for Vernalisation)
    CompetenceHS = 1.1 #
    
    # Calculate TSHS for each envronment set from FLNData
    data['TSHS_LV']        = TSHS(FLN_LV)
    data['TSHS_LN']         = TSHS(FLN_LN)
    data['TSHS_SV']         = TSHS(FLN_SV)
    data['TSHS_SN']          = TSHS(FLN_SN)
    
    # Photoperiod sensitivity (PPS)
                                 # the difference between TSHS at 8 and 16 h pp under 
                                 # full vernalisation.
    data['PPS']                = max(0,data['TSHS_SV'] - data['TSHS_LV'])
    
    # Maximum delta for Upregulation of Vrn3 (MaxDVrn3)
                                 # Occurs under long Pp conditions
                                 # Assuming Vrn3 increases from 0 - 1 between VS to TS and this takes 
                                 # 3 HS under long Pp conditions.
    data['MaxDVrn3']            = 1.0 / L_HSVsTs
    
    # Base delta for upredulation of Vrn3 (BaseDVrn3) 
                                 # Occurs under short Pp conditions
                                 # Assuming Vrn3 infreases from 0 - 1 from VS to TS 
                                 # and this take 3 HS plus the additional HS from short Pp delay.
    data['BaseDVrn3']          = 1.0 / (L_HSVsTs + data['PPS'])
    
   # Vernalistion Saturation Haun Stage under 8h Nil vern conditins  (VSHS_SN)
                                 # Determine how many HS it would take to express Vrn3 == 1.0 
                                 # and subtract this from TSHS_SNl.
                                 # Bound to CompetenceHS as Vernalisation won't occur before this.
    data['VSHS_SN']          = max(CompetenceHS,
                                     data['TSHS_SN'] - (L_HSVsTs + data['PPS']))
    
    # Base delta for upregulation of Vrn1 (BaseDVrn1) 
                                 # Occurs under non vernalising conditions
                                 # Assuming Vernalisation saturation occurs when Vrn1 == Vrn1Target 
                                 # under short Pp (no Vrn2) Vrn1Target = 1.0.
                                 # Need to include time from imbib to emerge in duration 
    data['BaseDVrn1']          = 1.0 / (data['VSHS_SN'] + data['EmergHS'])
    
    # Vernalisation Saturation Haun Stage under 16h Nil vern conditions (VSHS_LN)
                                 # Assuming Vern saturation occurs 3.0 HS before TSHS_LN
                                 # 3.0 is HS from VS to TS under long Pp conditions
                                 # Bound to CompetenceHS as Vernalisation won't occur before this.
    data['VSHS_LN']         = max(CompetenceHS,
                                     data['TSHS_LN'] - L_HSVsTs) 
    
    # Long day Increase in Vrn1Target (Vrn1TargetInc) 
                                 # The extention in VSHS due to the action of Vrn2 under long days 
                                 # Work out how much VSHS is delayed under long Pp conditions.
                                 # Multiply this by BaseDVrn1 to get how much more Vrn1 was 
                                 # expressed before vernalisation
    data['L_Vrn1TargetInc']    = (data['VSHS_LN'] - data['VSHS_SN']) * data['BaseDVrn1']
    
    # Maximum delta for upregulation of Vrn2 (MaxDVrn2)
                                 # Occurs under long day conditions
                                 # LDVrnTargetInc is divided by the HS duration of Vrn2 expression 
                                 # which goes from CompetenceHS to VS.
    data['MaxDVrn2']           = max(0.0,
                                     data['L_Vrn1TargetInc'] / (data['VSHS_LN'] - CompetenceHS))
    
    # Vernalistion Saturation Haun Stage under 8h Full vern conditions (VSHS_SV)
                                 # Assuming VSHS occurs 3 HS pluss the HS delay from short Pp prior
                                 # prior to TSHS
                                 # Bound to CompetenceHS as Vernalisation won't occur before this.
    data['VSHS_SV']         = max(CompetenceHS,
                                     data['TSHS_SV'] - (L_HSVsTs + data['PPS']))
    
   # Number of Haun stages from the end of treatment until Vernalisation saturation
                                 # Under 8 H full vern conditions (TransToVSHS_SV)
    data['TransToVSHS_SV']  = max(0.0,data['VSHS_SV'] + data['EmergHS'] - data['VernTreatHS'])
    
    # The amount of methalated Vrn1 at the time of transition from vern treatment (MethVern1@Trans)
                                 # Under 8h when VrnTarget will be 1.0
                                 # VrnTarget less the amount of Vrn1 that expressed after transition
    data['MethVern1@Trans']    = max(0,1.0 - data['TransToVSHS_SV'] * data['BaseDVrn1'])
    
    # BaseVer1 expressed at transition from vernalisation treatment
    data['BaseVrn1@Trans']     = min(1.0,data['VernTreatHS'] * data['BaseDVrn1'])
    
    # Methalated Cold upredulated Vrn1 expression at the time of transition (MethColdVrn1@Trans) 
                                 # Subtract out BasedVrn1 expression up to transition
    data['MethColdVrn1@Trans'] = max(0.0, data['MethVern1@Trans'] - data['BaseVrn1@Trans'])
    
    # The timing at which methalation started 
                                 # relative to transition from vernalising treatments (RelativeMethTiming)
    data['RelativeMethTiming'] = 1 - data['MethColdVrn1@Trans']
    
    # The amount of BaseVrn1 that had expressed at the time methalation of ColdVrn1 started
    data['BaseVern1@Meth']     = data['BaseVrn1@Trans'] * data['RelativeMethTiming']
    
    # Cold induced delta upregulation of Vrn1 at treatment temperature ('DVrn1@Tt')
                                 # Methalation of Cold upregulated Vrn1 occurs when 
                                 # ColdUpRegVrn1 > Vrn1Target
                                 # Vrn1Target will be 1.0 in 8 hour conditions
                                 # so ColdUpRegVrn1 at transition will be:
                                 # 1.0 - BaseVern1@Meth + MethColdVrn1@Trans.
                                 # divide by treatment HS duration to give rate
    data['DVrn1@Tt']            = (1.0 -  data['BaseVern1@Meth'] + 
                                   data['MethColdVrn1@Trans'])/data['VernTreatHS']
    
    # Maximum upregulation delta Vrn1 (MUdVrn1)
                                 # The rate of dVrn1/HS at 0oC.  Calculate by rearanging UdVrn1 equation
                                 # UdVrn1 = MUdVrn1 * np.exp(k*Tt) as UdVrn1@Tt is known 
    data['MaxDVrn1']            = data['DVrn1@Tt'] / np.exp(k * Tt)
    
    return data

Overwriting C:\Anaconda\Lib\CAMP.py
